In [1]:
!pip install minsearch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## Minsearch retrieval

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [18]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results = 5
)

## OpenAI implementation

In [ ]:
from openai import OpenAI

In [11]:
client = OpenAI()

In [13]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [{"role": "user", "content": q}]
)

In [14]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on the policies of the institution or platform offering the course. Many institutions have specific deadlines for enrollment, while others may allow late registration under certain circumstances. \n\nHere are some steps you can take:\n\n1. **Check the Course Policy**: Look for any information regarding late enrollment on the course webpage or syllabus.\n2. **Contact the Instructor or Admin**: Reach out to the course instructor or the administration office to inquire about the possibility of enrolling late.\n3. **Consider Alternatives**: If late enrollment isn't possible, ask if there are similar courses offered in the future or if there are resources available to catch up on missed content.\n\nIt's always best to act quickly as there may be limited opportunities for late enrollment."

In [15]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [19]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [22]:
prompt = prompt_template.format(question=q, context=context).strip()

In [24]:
response = client.chat.completions.create(
    model = 'gpt-4o-mini',
    messages = [{"role": "user", "content": prompt}]
)

response.choices[0].message.content

"Yes, you can still enroll in the course even after it has started. You are eligible to submit the homeworks, but keep in mind that there will be deadlines for turning in the final projects, so it's advisable not to leave everything until the last minute."

## Cleaner and modular

In [25]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [26]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [27]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [28]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
query = 'how do I run kafka?'

In [30]:
rag(query)

'To run Kafka with a Java producer, you should navigate to your project directory and execute the following command in the terminal:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nIf you are using Python, ensure you have created a virtual environment, installed the necessary packages, and then you can run your Kafka producer or consumer within that environment.'

In [31]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. You are eligible to submit the homeworks even if you didn't register. However, remember that there will be deadlines for the final projects, so it's important not to leave everything until the last minute."

## Implementing ElasticSearch

In [32]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [33]:
from elasticsearch import Elasticsearch

In [34]:
es_client = Elasticsearch('http://localhost:9200') 

In [35]:
es_client.info()

ObjectApiResponse({'name': 'c61c5781cf35', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'dbgNzGfLRNq-HUvg0p8yFg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [36]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [37]:
from tqdm.auto import tqdm

In [38]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [39]:
query = 'I just disovered the course. Can I still join it?'

In [40]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [41]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [42]:
rag(query)

"Yes, you can still join the course even after the start date. While you don't need to register, you are eligible to submit homework. However, be mindful of the deadlines for turning in the final projects to avoid last-minute issues."